In [ ]:
#!pip install basemap
#!pip install basemap-data
#!pip install folium
#!pip install kneed

This notebook investigates police crime statistics that collect street crimes reported by area for each crime type in England and Wales. 

#### Crime types

[Crime Tree](https://assets-hmicfrs.justiceinspectorates.gov.uk/uploads/crime-tree.pdf)

* Violence and sexual offences - homicide, violence with and without injury, rape and other sexual offences
* Robbery - of business and personal property
* Theft - burglary in dwelling or other building, vehicles, person, bicycle, shoplifting, all others
* Criminal damage and arson
* Drugs - trafficking and possession
* Possession of weapons
* Public order
* Other miscellaneous offences

#### Data Sources:

[Crime Figures](https://data.police.uk/data/)

[Population Figures](https://cy.ons.gov.uk/peoplepopulationandcommunity/crimeandjustice/datasets/policeforceareadatatables)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from mpl_toolkits.basemap import Basemap
import folium
from folium import plugins
from sklearn.cluster import KMeans

In [ ]:
# Set path to get data

paths = ['2023-01','2023-02','2023-03','2023-04','2023-05','2023-06','2023-07',
         '2023-08','2023-09','2023-10','2023-11','2023-12']

# Create empty file list
file_list = []
names = []

# Iterate over paths and add to the file list

for p in paths:
    path = r'C:\\Users\\imoge\\Downloads\\PoliceData\\' + p
    all_files = glob.glob(path + "/*.csv")
    file_list.append(all_files)
    for item in all_files:
        names.append(item.rsplit('.', 1)[0].rsplit('-',1)[0].rsplit('-',1)[1])
        names = sorted(names)
        
#  Convert to one list
all_files = [item for sublist in file_list for item in sublist]
len(all_files)

In [ ]:
# Create list of all files

li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
    
len(li)

In [ ]:
# Concat into a dataframe and get the shape
df = pd.concat(li, axis=0, ignore_index=True)
df.shape

In [ ]:
# Get the unique police units
df['Falls within'].unique()

In [ ]:
len(df['Falls within'].unique())

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

We have some missing data for several of the features including the crime ID, longitude, latitude, LSOA code and name and the last outcome category

In [ ]:
# Look at which areas have nulls for LSOA code
df[df['LSOA code'].isnull()]['Reported by'].value_counts()

In [ ]:
# Look at Northern Ireland
df[df['Reported by'] == 'Police Service of Northern Ireland'].head()

There is quite a lot of missing data so we will drop these

In [ ]:
df = df[df['Reported by']!= 'Police Service of Northern Ireland']
df.shape

In [ ]:
df[df['LSOA code'].isnull()].head()

The rows with no location details seem to coincide with no LSOA code or name so we will drop those rows

In [ ]:
df.dropna(subset = ['LSOA name'], inplace = True)

The remaining missing values are involved with the last outcome category, so if there is no outcome then there is no ID. We will drop both those columns

In [ ]:
df.drop(columns = ['Crime ID','Last outcome category', 'Context'],axis =1, inplace = True)

In [ ]:
df.shape

In [ ]:
df.head()

We will also drop the 'Reported by' category, the Location and the LSOA name

In [ ]:
df.drop(columns = ['Reported by','LSOA name'], axis = 1, inplace = True)
df.head()

In [ ]:
# Change the name of the Month column to date and then replace with a datetime object 
df.rename(columns = {'Month':'Date'},inplace = True)
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
# Extract year, month and day
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Month Name'] = df['Date'].dt.month_name()

In [ ]:
df.drop(columns = ['Date'], axis = 1, inplace = True)

In [ ]:
df.head()

## Exploratory Data Analysis

The highest number of reported crimes are unsurprisingly in the metropolitan police area. We can add in some population figures to allow comparison between areas

### Where are the crimes being reported to occur?

In [ ]:
# Map these crimes on a map of England and Wales

lat = df['Latitude'].to_list()
long = df['Longitude'].to_list()

crime_map = folium.Map([52.3555, 1.1743], zoom_start=5)

heatmap = plugins.HeatMap(list(zip(lat,long)),radius = 1, blur = 1)
crime_map.add_child(heatmap)

In [ ]:
file_name = 'C:/Users/imoge/Projects'
crime_map.save(file_name + '.html')

In [ ]:
# Plot crimes by area
sns.set_palette('dark')
fig, ax = plt.subplots(figsize = (8,10))
ax = df['Falls within'].value_counts().plot(kind = 'barh')
plt.title('Crimes by Area 2023')
ax.invert_yaxis()
ax.spines[['right', 'top']].set_visible(False);

Most crimes are reported in the area of the Metropolitan Police and the least by the City of London police

In [ ]:
# Plot crimes by type
sns.set_palette('dark')
ax = df['Crime type'].value_counts().plot(kind = 'barh')
plt.title('Crimes by Type 2023')
ax.spines[['right', 'top']].set_visible(False);

Most crimes involve violence and sexual offences

In [ ]:
# Find the maximum value for each crime type
maxValueIndex = crime_pivot.idxmax()
 
print("Maximum values of columns are at row index position :")
print(maxValueIndex)

With the exception of possession of weapons the Metropolitan Police Service has the highest number of reported crimes of all types

### When do crimes occur?

In [ ]:
df['Month Name'].value_counts().plot(kind = 'bar')

Most in June and least in December

### Adding in population figures

To make proper comparisions we need to add some population figures

In [ ]:
# Bring in the population figures
pop = pd.read_excel(r'C:\\Users\\imoge\\Downloads\\PolicePop.xlsx',sheet_name = 5, skiprows = 7, usecols = "A,B,C")
pop.columns = ['Area Code','Area Name', 'Pop']
pop.shape

In [ ]:
pop.sort_values(by = 'Area Name')

In [ ]:
# Drop rows we don't need
drop_these = ['E92000001','K04000001','E12000006','E12000003','W92000004','E12000009', 'E12000008','E12000007',
              'E12000005','E12000004','E12000001','E12000002']


pop = pop.loc[~pop['Area Code'].isin(drop_these)]

pop.shape

In [ ]:
pop.sort_values(by = 'Area Name')

We want to merge this population data with the crime data. As we only have the higher area code for population, we will need to merge on the area name and we need to make sure the names in one table match the other.

In [ ]:
by_county = df.groupby('Falls within', as_index = False)['Year'].count()
by_county

In [ ]:
# Add join column to each dataframe
by_county['join']= 1
pop['join'] = 1

In [ ]:
# Mark each row where the Area Name is found within the Falls Within column of the by_county dataframe

merged_df = by_county.merge(pop, on = 'join')
pop.drop('join',axis = 1, inplace = True)

merged_df['match'] = merged_df.apply(lambda x: x['Falls within'].find(x['Area Name']), axis = 1).ge(0)
merged_df

In [ ]:
# Select the rows and columns we want
matched_df = merged_df[merged_df['match']==True]
matched_df = matched_df[['Falls within','Pop']]
matched_df.shape

In [ ]:
matched_df

We are missing Greater Manchester police for which we do not have any police reported crime figures and British Transport Police for which we wouldn't have population figures. We will exclude these from the analysis going forward

In [ ]:
df = df[df['Falls within']!='British Transport Police']

With the exception of possession of weapons, the highest number of crimes of all types are reported in the area of the Metropolitan Police Service

We should compare the crimes per head or the crimes per thousand in order to properly compare these

### Crimes per 1000 population

In [ ]:
# Groupby the area and crime time and then count the reported crimes, then creating a pivot table with a total column
crime_pivot = df.groupby(['Falls within','Crime type'],as_index = False)['LSOA code'].count().pivot(index = 'Falls within', 
                                                                                     columns = 'Crime type', 
                                                                                     values = 'LSOA code')
crime_pivot['Total'] = crime_pivot.sum(axis = 1)
crime_pivot

In [ ]:
# Create a list of area names
count_names = list(matched_df['Falls within'])

In [ ]:
# Merge the pivot with the population figures, then calculate number of crimes per 1000 population, add the index name and sort
crime_pivot_merged = crime_pivot.merge(matched_df, left_on = crime_pivot.index, right_on = 'Falls within').drop(columns = ['Falls within'],axis = 1)
crime_per_thousand = crime_pivot_merged.iloc[:,0:-1].apply(lambda x: x/crime_pivot_merged['Pop']*1000)
crime_per_thousand.index = count_names
crime_per_thousand.sort_values(by = 'Total',ascending = False)

In [ ]:
# Plot the crimes per 1000 population
fig, ax = plt.subplots(figsize = (8,12))
crime_per_thousand[['Total']].sort_values(by = 'Total',ascending = False).plot(ax = ax, kind = 'barh')
plt.title('Crimes per 1000 Population by Area 2023')
ax.invert_yaxis()
ax.spines[['right', 'top']].set_visible(False);

We can see some changes in the chart when we calculate the crimes per 1000 population. The City of London was at the bottom of the table in terms of the number of reported street crimes but at the top when calculated per 000 head of population

### Correlation between crime types

In [ ]:
# Look at correlations
crime_corr = crime_per_thousand.iloc[:,0:-1].corr()
crime_corr

This is quite hard to look at so we will do a heatmap

In [ ]:
# Lets look at a heatmap
cmap = sns.cm.rocket_r
fig, ax = plt.subplots(figsize = (12,10))
sns.heatmap(crime_corr, annot = True, cmap = cmap);

There seems to be positive correlation amongst most of the crime types and it is easier to see which ones are not highly correlated to other crimes - vehicle and anti-social behaviour

In [ ]:
# Get correlation matrix, remove duplicates and unstack
corr_mat = crime_corr
upper_corr_mat = corr_mat.where( 
    np.triu(np.ones(corr_mat.shape), k=1).astype(bool)) 
unique_corr_pairs = upper_corr_mat.unstack().dropna() 
  
# Sort correlation pairs 
sorted_mat = unique_corr_pairs.sort_values() 
sorted_mat = pd.DataFrame(sorted_mat).sort_values(by = 0,ascending = False)
sorted_mat = sorted_mat[sorted_mat[0]<1.0]
sorted_mat.head(10)

The highest correlations are various types of theft which are all interelated in terms of occurrence

In [ ]:
sorted_mat.tail()

In [ ]:
sns.pairplot(data = crime_per_thousand.iloc[:,0:-1])

The pairplot is a little difficult to see but the correlations indicate that the occurrence of one type of crime is highly correlated with another

In [ ]:
# Have a look at the highest correlation relationship
ax = sns.scatterplot(data = crime_per_thousand, x = crime_pivot['Theft from the person'], y = crime_pivot['Other theft'])
plt.title('Other theft against theft from the person')
plt.xlabel('Theft from the person', labelpad = 10)
ax.spines[['right', 'top']].set_visible(False);

* This seems to suggest a strong correlation between theft from the person and other theft occurring in an area
* There is one point way above the rest, which relates to the figures for the Metroplolitan Police area

### Crimes in the Metropolitan Area

Lets have a look at all the incidences on a scatter plot for that police area which is London

In [ ]:
# Get location by crime details
crime_met = df[df['Falls within'] == 'Metropolitan Police Service']
location_met = crime_met[['Latitude','Longitude','Crime type']]
location_met = location_met.dropna()
#location_met = location_met.loc[(location_met['Latitude']<52) & (location_met['Longitude'] > -2)]  

# Plot the metropolitan data
fig, ax = plt.subplots(figsize = (11,8))
ax = sns.scatterplot(data = location_met, x = 'Longitude', y = 'Latitude', hue = 'Crime type')

# Legend
handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels),loc='upper right', bbox_to_anchor=(1.25, 1),
          ncol=1, fancybox=True, shadow=True)
plt.title('Crimes by Latitude and Longitude for the Metropolitan Police Area');

In [ ]:
# Plotting heat map of all crimes
lat = crime_met_map['Latitude'].to_list()
long = crime_met_map['Longitude'].to_list()

crime_map = folium.Map([51.5072,  0.1276], zoom_start=10)

heatmap = plugins.HeatMap(list(zip(lat,long)),radius = 2, blur = 1)
crime_map.add_child(heatmap)

Lets drop out the violent crime

In [ ]:
# Focussing on robberies and theft
exclude_crimes = ['Violence and sexual offences', 'Possession of weapons', ' Public order', ' Vehicle crime',
                 'Other crime','Drugs','Criminal damage and arson', 'Anti-social behaviour']

# Get location by crime details
crime_met_exclude = df[(df['Falls within'] == 'Metropolitan Police Service') & (~df['Crime type'].isin(exclude_crimes))]
location_met = crime_met_exclude[['Latitude','Longitude','Crime type']]
location_met = location_met.dropna()
location_met = location_met.loc[(location_met['Latitude']<51.7) & (location_met['Longitude'] > -0.5) & (location_met['Longitude'] < 0.3)& (location_met['Latitude']>51.3)]

# Plot the metropolitan data
fig, ax = plt.subplots(figsize = (11,8))
ax = sns.scatterplot(data = location_met, x = 'Longitude', y = 'Latitude', hue = 'Crime type')

# Legend
handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels),loc='upper right', bbox_to_anchor=(1.25, 1),
          ncol=1, fancybox=True, shadow=True)
plt.title('Theft, Burglary and Robbery Crimes for the Metropolitan Police Area');

* The incidents are very much clustered in Central London.
* The majority of incidents seems to be robbery.
* We can see a cluster of theft from the person in the very centre

We could analyse it further by seeing which LSOA code has the most crimes reported in the area and what crimes took place

In [ ]:
# Group crimes by LSOA code to find where most take place
crime_met.groupby(['LSOA code'],as_index = False)['Crime type'].count().sort_values(by = 'Crime type',ascending = False).head()

We can see two areas with the highest number of recorded incidents

In [ ]:
# Get those rows in these top 3 areas
codes = ['E01035716','E01004734']
met_codes = crime_met[crime_met['LSOA code'].isin(codes)]

In [ ]:
# Group by code and crime type and pivot
met_codes.groupby(['LSOA code','Crime type'],as_index = False)['Year'].count().pivot(index = 'Crime type',
                                                                                     columns = 'LSOA code',
                                                                                    values = 'Year')

* We can see that theft is the most reported type of crime in these two areas is theft from the person and other theft
* Lets have a look at violence and sexual offences which is perhaps considered to be the most serious of these crimes
* The most violent crimes are in LSOA code E01004734

In [ ]:
# In what location?
viol = crime_met[(crime_met['LSOA code']=='E01004734') & (crime_met['Crime type']=='Violence and sexual offences')].groupby('Location',
                                                                                                                     as_index = False)['Year'].count().sort_values(by = 'Year', 
                                                                                                                     ascending = False).head(10)
viol

* We can see that by narrowing the dataframes down we can identify where the most  offences occur and in what location in that LSOA area

* We can see that the nightclub and theatre/concert hall are key locations

In [ ]:
# Filter for the LSOA into a dataframe
E01004734_viol = crime_met[(crime_met['LSOA code']=='E01004734') & (crime_met['Crime type']=='Violence and sexual offences')]

In [ ]:
# Draw map of violence offences in the metropolitan LSOA E01004734
lat = E01004734_viol['Latitude'].to_list()
long = E01004734_viol['Longitude'].to_list()

E01004734_map = folium.Map([51.51,  -0.122], zoom_start=14.5)

heatmap = plugins.HeatMap(list(zip(lat,long)),radius = 5, blur = 1)
E01004734_map.add_child(heatmap)

These offences are being reported to occur in or around Leicester Square, a nightlife area

In [ ]:
# What month are these being reported?
E01004734_viol['Month'].value_counts().plot(kind = 'bar')

Most are reported in December and the least in March and April

### Crimes in the City of London

We found earlier that the City of London had the highest reported crimes per 1000 population.

In [ ]:
crime_city = df[df['Falls within'] == 'City of London Police']

In [ ]:
crime_city.groupby(['Crime type'],as_index = False)['Year'].count().sort_values(by = 'Year',ascending = False)

We can see that other theft, violence and sexual offences and theft from the person are the most common types of crime for this area. 

In [ ]:
# Group crimes by LSOA code to find where most take place
crime_city.groupby(['LSOA code'],as_index = False)['Crime type'].count().sort_values(by = 'Crime type',ascending = False).head()

* We can see that offences are very much concentrated in one LSOA
* Where are the violent crimes happening in the City of London?

In [ ]:
# Filter for violent and group by LSOA code
city_viol = crime_city[crime_city['Crime type'] == 'Violence and sexual offences']
city_viol.groupby('LSOA code',as_index = False)['Year'].count()

The same LSOA code appears, so again we can find out the top locations

In [ ]:
city_viol.groupby('Location',as_index = False)['Year'].count().sort_values(by = 'Year',ascending = False)

In [ ]:
# Get the violent crime for this LSOA 
E01032739_viol = city_viol[(city_viol['LSOA code']=='E01032739')]

In [ ]:
# Draw map of violence offences in the metropolitan LSOA E01004734
lat = E01032739_viol['Latitude'].to_list()
long = E01032739_viol['Longitude'].to_list()

E01004734_map = folium.Map([51.51,  -0.09], zoom_start=14)

heatmap = plugins.HeatMap(list(zip(lat,long)),radius = 5, blur = 1)
E01004734_map.add_child(heatmap)

## Can we cluster the data to find areas in common for crimes?

In [ ]:
# Select the raw crime figures
df_cluster = crime_pivot.iloc[:,0:-1]
df_cluster.head()

In [ ]:
# Scale the datapoints for use with clustering model
from sklearn.preprocessing import StandardScaler
object = StandardScaler()
standard = object.fit_transform(df_cluster)
standard

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = '1'

In [ ]:
# Plot the elbow curve

plt.figure(figsize = (6,4))
wcss = []
for i in range(1,11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(standard)
    wcss.append(kmeans.inertia_)
    
plt.plot(range(1,11), wcss, 'bx-')
plt.title('Elbow Method')
plt.xlabel('Num clusters')
plt.ylabel('WCSS')

In [ ]:
# Plot the point where the curve starts to flatten
from kneed import KneeLocator
y = wcss
x = range(1, len(y)+1)
kn = KneeLocator(x, y, curve='convex', direction='decreasing')
print(kn.knee)

plt.plot(range(1,11), wcss, 'bx-')
plt.title('Elbow Method')
plt.xlabel('Num clusters')
plt.ylabel('WCSS')

plt.vlines(kn.knee, plt.ylim()[0], plt.ylim()[1], linestyles='dashed');

In [ ]:
# Run kmeans with 3 clusters
kmeans = KMeans(n_clusters = 3, init = 'k-means++', random_state = 42)
mod1 = kmeans.fit_predict(standard)
mod1 = mod1+1
mod1

In [ ]:
# Create cluster column
cluster = list(mod1)
df_cluster['Cluster']=cluster
df_cluster.head()

In [ ]:
# Have a look at the means
mean_cluster = pd.DataFrame(round(df_cluster.groupby('Cluster').mean()))
mean_cluster

We can see that the mean values for crimes vary by cluster 

In [ ]:
# Plot these
ax = mean_cluster.plot(kind = 'bar', stacked = True)

# Legend
handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels),loc='upper right', bbox_to_anchor=(1.6, 1),
          ncol=1, fancybox=True, shadow=True);


In [ ]:
# Reset index and rename the area column from index
df_cluster.reset_index(inplace = True)
df_cluster.rename(columns = {'index':'Falls within'}, inplace = True)
df_cluster.head()

In [ ]:
# Group the areas by cluster
df_cluster.groupby('Cluster', as_index = False)['Falls within'].count()

In [ ]:
# What areas are in cluster 1
df_cluster[df_cluster['Cluster']==1]

In [ ]:
# What areas are in cluster 2
df_cluster[df_cluster['Cluster']==2]

In [ ]:
# What areas are in cluster 3
df_cluster[df_cluster['Cluster']==3]

In [ ]:
# Plot a few charts of crimes to see the clusters
sns.scatterplot(data = df_cluster, x = 'Robbery', y = 'Shoplifting', hue = 'Cluster')
plt.title('Shoplifting against Robbery');

In [ ]:
sns.scatterplot(data = df_cluster, x = 'Robbery', y = 'Burglary', hue = 'Cluster')
plt.title('Burglary against Robbery');

In [ ]:
sns.scatterplot(data = df_cluster, x = 'Violence and sexual offences', y = 'Robbery', hue = 'Cluster')
plt.title('Violence against Robbery');

Building a model based on raw crime figures by area provides a straightforward representation of the total number of crimes reported in each area. This approach can be useful for identifying areas with the highest number of reported crimes, without considering the population size. It can help in understanding the overall crime distribution across different areas.

We can see that the clusters are quite distinct with lower crime areas clustered as the bottom left of the charts and these seem to be mostly shire or more rural counties. The Metropolitan Police area is an outlier of one point in the cluster up to the top right. The other two clusters are more towards cluster one as the left to middle bottom of the chart. This probably mostly reflects differences in the types of areas, being more urban, with higher populations which tend to experience higher crimes. 

Building a model based on crime per head (crime rate) can provide a more normalized view of crime statistics, taking into account the population size of each area. This can help in comparing areas with different population sizes and identifying areas with higher crime rates relative to their population. It can be useful for understanding the relative impact of crime on a per capita basis.

We will rerun the model with crime per capita figures to see if we can see any changes

In [ ]:
# Select the raw crime figures
df_cluster2 = crime_per_thousand.iloc[:,0:-1]

In [ ]:
# Scale the datapoints for use with clustering model
object2 = StandardScaler()
standard = object2.fit_transform(df_cluster2)
standard

In [ ]:
# Plot the elbow chart with the change point

plt.figure(figsize = (6,4))
wcss = []
for i in range(1,11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(standard)
    wcss.append(kmeans.inertia_)
    
y = wcss
x = range(1, len(y)+1)
kn = KneeLocator(x, y, curve='convex', direction='decreasing')
print(kn.knee)

plt.plot(range(1,11), wcss, 'bx-')
plt.title('Elbow Method')
plt.xlabel('Num clusters')
plt.ylabel('WCSS')

plt.vlines(kn.knee, plt.ylim()[0], plt.ylim()[1], linestyles='dashed');


Looks like the elbow is more at point 2 than 3

In [ ]:
# Run kmeans with 2 clusters
kmeans = KMeans(n_clusters = 2, init = 'k-means++', random_state = 42)
mod2 = kmeans.fit_predict(standard)
mod2 = mod2+1
mod2

In [ ]:
# Add the clusters to the dataframe
cluster2 = list(mod2)
df_cluster2['Cluster'] = cluster2

In [ ]:
# Get the means for each cluster
mean_cluster2 = pd.DataFrame(round(df_cluster2.groupby('Cluster').mean()))
mean_cluster2

In [ ]:
# Plot the means
ax = mean_cluster2.plot(kind = 'bar', stacked = True)

# Legend
handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels),loc='upper right', bbox_to_anchor=(1.6, 1),
          ncol=1, fancybox=True, shadow=True);

In [ ]:
# Reset the index and rename, then group the areas by cluster

df_cluster2.reset_index(inplace = True)
df_cluster2.rename(columns = {'index':'Falls within'}, inplace = True)
df_cluster2.head()

df_cluster2.groupby('Cluster', as_index = False)['Falls within'].count()

In [ ]:
# What areas are in cluster 1
df_cluster2[df_cluster2['Cluster']==1]

This time instead of the Metropolitan Police in a distinct cluster by itself, we have the City of London and everything else is in the second cluster

In [ ]:
# Plot an example chart of crimes to see the clusters
sns.scatterplot(data = df_cluster2, x = 'Robbery', y = 'Shoplifting', hue = 'Cluster')
plt.title('Shoplifting against Robbery');

#### Conclusions

* After using the crime per capita figure to cluster the data and thereby controlling for the effect of population, we can see that one authority, the City of London has much higher crime figures than the others. 
* This is a simplistic analysis, as in fact we might choose to add in a whole range of other socio-economic factors that have been found to be linked to crime in research papers and try to cluster on these. This would allow us to group police areas together on common characteristics to try to see what factors have the most influence on crime. 
* We need to look at per capita crime to compare and that the City of London has by far and away the highest number of crimes per head of people living there. 
* Even so, we know that this area has a huge transient population of daily visitors for work etc and just using the population living there is perhaps misleading. Further work would need to be undertaken to uncover the true drivers of crime in this area.